# LSH

python implementation of LSH

dataset

4 articles pulled from CNN

concatenation of 3 of those, this one artificialy produce some high similarity scores

outline

## Part I: computing similarity amongst text documents
1. Introduce a shingle function. Clean and split each text file into a set of K-shingles
2. Compute the exact Jaccard similarity (intersection over union) between all pairs
3. Create and apply a MinHashing class:
    1. Initialize with a dictionary of key-value pairs for the shingles
    2. Apply "universal hashing" to perform minhashing on a shingle set
    3. can be called like a function to compute a **signature matrix**
4. Evaluate MinHashing effectiveness by computing scores of all pairs
5. Introduce LSH for finding **candidate pairs**, i.e. use a banded signature matrix to find all pairs whose similarity is likely above a threshold
6. Make this efficient, by using hash table for band, column ids, allowing O(n) comparison

## Part II: computing similarity amongst vectors

* Afterwards, we provide an additional LSH family for Euclidean spaces, namely cosine similarity. 
* This is used to ascertain the similarity of vectors in a D-dimensional space.
* Can be implemented using the *Random Hyperplanes* hashing method.

In [10]:

%config Completer.use_jedi = False

import os
import time
import itertools
import collections
import numpy as np
from os.path import join as PJ
# import matplotlib.pyplot as plt

# Part I: computing similarity amongst text documents

## Shingling

1. Create k shingles of for each document
2. Calculate total shingles,
3. Conaider Input Matrix (Shingles x documents) -  $I_{S \times D}$
4. Create MinHasher(the minsh hash object)
6. Hash the document into Signature Matrix - $M_{\alpha \times D}$

where $\alpha$ is the `n_signature`, you should make $\alpha << S$

Because we do min-hashing do preserve the similarity of Jaccard Similarity
and do the dimension reduction 

In [3]:

HOME = os.getcwd()
TARGET = os.path.join(HOME,'data', 'sampledocs_jaccard')

print(HOME,TARGET, sep='\n')

documents = []
for article in os.listdir(TARGET):
    if article == 'stopwords':
        continue
    path = os.path.join(TARGET, article)
    with open(path, 'r') as file:
        documents.append(file.read())
        
stopwords = []
with open(os.path.join(TARGET, 'stopwords'), 'r') as file:
    for line in file:
        stopwords.append(line.strip())
        
for i, doc in enumerate(documents):
    doc = doc.strip().replace('\n', ' ').lower()
    for word in stopwords:
        doc = doc.replace(' '+word+' ', ' ')
    documents[i] = doc

print(f"Average char-length: \
{np.mean(np.array([len(x) for x in documents]))}")
print(f"Min char-length: {min(len(x) for x in documents)}")
print(f"Max char-length: {max(len(x) for x in documents)}")

/home/joetsai/work/yulong/cs246_mining_massive_datasets/implementation/lsh
/home/joetsai/work/yulong/cs246_mining_massive_datasets/implementation/lsh/data/sampledocs_jaccard
Average char-length: 3651.6
Min char-length: 2412
Max char-length: 5873


In [4]:
# create K-shingles by sliding window approach
# TODO [joe] - make this sparse matrix
def getShingles(str1, K=5):
    d1 = set()
    for i in range(len(str1)-K):
        d1.add(str1[i:i+K])
    print(f"Found {len(d1)} unique shingles, out of {len(str1)} possible.")
    return d1
doc_shingles = [getShingles(s, 5) for s in documents]

Found 2782 unique shingles, out of 3673 possible.
Found 2091 unique shingles, out of 3291 possible.
Found 1918 unique shingles, out of 2412 possible.
Found 3953 unique shingles, out of 5873 possible.
Found 2060 unique shingles, out of 3009 possible.


In [5]:
def jaccardSim(d1,d2):
    return len(d1.intersection(d2))/len(d1.union(d2))

# itertools.combinations finds all (,n) n-pairs
# then we use a map op on the tuples with jaccardSim
pairs = itertools.combinations(documents, 2)
pair_labels = []
pair_sims = []
for x1, x2 in itertools.combinations(zip(range(len(doc_shingles)),doc_shingles), 2):
    pair_labels.append((x1[0],x2[0]))
    pair_sims.append(jaccardSim(x1[1],x2[1]))
    
print(f"**~~~~~~ True similarity scores ~~~~~~**")
print("Pair\tScore")
print("-"*14)
for pair, score in zip(pair_labels, pair_sims):
    print(f"{pair}\t{score:.3f}")

**~~~~~~ True similarity scores ~~~~~~**
Pair	Score
--------------
(0, 1)	0.081
(0, 2)	0.052
(0, 3)	0.083
(0, 4)	0.069
(1, 2)	0.051
(1, 3)	0.294
(1, 4)	0.093
(2, 3)	0.400
(2, 4)	0.050
(3, 4)	0.336


In [7]:
# Take union of all sets. Convert to an array and assign
# each element an integer based on position in array
fullset = set.union(*doc_shingles)
shingle_dict = dict(zip(list(fullset),range(len(fullset))))
print(f"There are {len(shingle_dict)} shingles")

There are 7534 shingles


In [70]:
doc_shingles[0]

{'far a',
 'e maj',
 'ecede',
 'reats',
 'onall',
 'd the',
 'olora',
 's. pa',
 'ies, ',
 'ult e',
 'oaths',
 'in. w',
 'withi',
 'f. fe',
 ' prep',
 'edenc',
 'r arg',
 's, pe',
 'lecte',
 'kgrou',
 ' trum',
 'udici',
 'irmly',
 'p gon',
 'hilli',
 'fice ',
 'imed ',
 'us, t',
 "'s vi",
 'rivol',
 ' blat',
 's ben',
 'd tv ',
 'ane u',
 'ness,',
 ' powe',
 'atitu',
 'date ',
 ' one ',
 'own e',
 '" cou',
 'vote ',
 's cha',
 'ng co',
 's too',
 ', dis',
 'mselv',
 'feroc',
 'fully',
 ' floo',
 'rges ',
 'far, ',
 'epost',
 'aimed',
 ', als',
 'order',
 'crede',
 ' back',
 'es la',
 'micus',
 'ults,',
 'gious',
 'litic',
 'ent, ',
 'ieved',
 'ry pe',
 'use f',
 'led w',
 'w oat',
 'ure w',
 '- cou',
 'g, re',
 'may s',
 'ures ',
 'ected',
 'serva',
 ' prim',
 'oney ',
 'y tra',
 't.alm',
 'ember',
 'normo',
 '. tru',
 's sta',
 'here ',
 'ns ju',
 'epubl',
 'rged ',
 'ent-e',
 'onal ',
 'eful,',
 'e fri',
 'assau',
 'e law',
 'anyon',
 'untry',
 'ed jo',
 'n cre',
 'profa',
 'den l',


In [29]:
# shingle = 5
# shingle_dict
# len(doc_shingles[0])
############ Examples #############
# https://numpy.org/doc/1.20/reference/generated/numpy.dot.html?highlight=dot#numpy.dot

# a @ b means a, b are two dimentional matrix,
# and the result is the same as np.dot(a,b)

a = np.array([[1, 0], [0, 1]])
b = np.array([[4, 1], [2, 2]])


print(
    'shingle of doc 1 example : ',
    list(doc_shingles[0])[:5],
    np.dot(a,b),
    a @ b,
    sep='\n'
)

shingle of doc 1 example : 
['far a', 'e maj', 'ecede', 'reats', 'onall']
[[4 1]
 [2 2]]
[[4 1]
 [2 2]]


In [72]:
# Create a hash function
# define as a callable class, so that we only initialize 
# random function once
# This is only Minhasher

class HashManager():
    def __init__(self, shingle_dict : dict, n_signature : int) -> None:
        self.shingle_dict = shingle_dict
        self.N = len(shingle_dict)
        self.n_signature = n_signature
        # create random integer from zero to N
        # TODO
        self.params = np.random.randint(self.N,
                                        size=[n_signature, 2])
    
    def _permuteRow(self, row):
        # TODO
        return self.params @ np.array([1, row]) % self.N
    
    def __call__(self, docs):
        
        # initialize signature matrix
        sig = np.full(
            (self.n_signature, len(docs)),
             np.inf
            )
        
        # each doc in docs is assumed to be an iterable object
        # we hash all the docs into signature matrix
        for doc_i, doc in enumerate(docs):
            for shingle in doc:
#                 print(list(doc)[:5], shingle)
                orig_row = shingle_dict[shingle]
                curr_col = self._permuteRow(orig_row)
                sig[:, doc_i] = np.minimum(
                    sig[:, doc_i],
                    curr_col
                )
        return sig.astype(int)
    
# run some tests

try:
    n_signature = 100
    print("Initilization")
    min_hasher = HashManager(shingle_dict,
                             n_signature=n_signature)
    print("passed")
    
    print('parametets is the right shape')
    assert(min_hasher.params.shape == (n_signature, 2))
    print('passed')
    
    print("permuting a row integer returns array")
    curr_col = min_hasher._permuteRow(3)
    assert(curr_col.shape == (n_signature, ))
    print('passed')
    
    print("Compute minhashed signature matrix : ")
    res = min_hasher(doc_shingles)
    assert res.shape == (n_signature, len(doc_shingles))
    print("passed")
    
except Exception as e:
    print(e.args)

Initilization
passed
parametets is the right shape
passed
permuting a row integer returns array
passed
Compute minhashed signature matrix : 
passed


In [73]:
def trueSimScores(doc_shingles):
    pair_labels = []
    pair_sims = []
    idxs = range(len(doc_shingles))
    for x1, x2 in itertools.combinations(zip(idxs,doc_shingles), 2):
        pair_labels.append((x1[0], x2[0]))
        pair_sims.append(jaccardSim(x1[1], x2[1]))
    return dict(zip(pair_labels, pair_sims))
    
def sigSimScores(sig_mat):
#     cols = [sig_mat[:,i] for i in range(sig_mat.shape[1])]
    cols = sig_mat.T
    idxs = range(sig_mat.shape[1])
    
    pair_labels = []
    pair_sims = []
    for (i,col1), (j,col2) in itertools.combinations(zip(idxs, cols),2):
        pair_labels.append((i,j))
        pair_sims.append(np.mean(col1==col2))
    
    return dict(zip(pair_labels, pair_sims))

def printScoreComparison(true_dict, approx_dict):
    print(f"**~~~~~~ Similarity score comparison ~~~~~~**")
    print("Pair\t\tApprox\t\tTrue\t\t%Error")
    for pair, true_value in true_dict.items():
        approx_value = approx_dict[pair]
        err = 100*abs(true_value-approx_value)/true_value
        print(f"{pair}\t\t{approx_value:.3f}\t\t{true_value:.3f}\t\t{err:.2f}")

def candidatePairs(score_dict, threshold):
    return set(pair for pair, scr in score_dict.items() if scr>=threshold)

def accMatrix(true_dict, approx_dict, threshold):
    true_pairs = candidatePairs(true_dict, threshold)
    approx_pairs = candidatePairs(approx_dict, threshold)
    false_negatives = len(true_pairs - approx_pairs)
    false_positives = len(approx_pairs - true_pairs)
    print(f"False negatives: {false_negatives}")
    print(f"Potential false positives: {false_positives}")


In [80]:
min_hasher = HashManager(shingle_dict,
                             n_signature=20)

sig_mat = min_hasher(doc_shingles)
true_score_dict = trueSimScores(doc_shingles)
approx_score_dict = sigSimScores(sig_mat)
printScoreComparison(true_score_dict, approx_score_dict)

print("True pairs:",candidatePairs(true_score_dict, 0.25))
print("Candidate pairs:",candidatePairs(approx_score_dict, 0.25))
accMatrix(true_score_dict, approx_score_dict, 0.4)



**~~~~~~ Similarity score comparison ~~~~~~**
Pair		Approx		True		%Error
(0, 1)		0.200		0.081		147.01
(0, 2)		0.200		0.052		286.93
(0, 3)		0.300		0.083		261.57
(0, 4)		0.200		0.069		190.38
(1, 2)		0.200		0.051		289.08
(1, 3)		0.350		0.294		18.85
(1, 4)		0.200		0.093		114.52
(2, 3)		0.350		0.400		12.54
(2, 4)		0.050		0.050		0.24
(3, 4)		0.450		0.336		33.84
True pairs: {(3, 4), (1, 3), (2, 3)}
Candidate pairs: {(3, 4), (0, 3), (1, 3), (2, 3)}
False negatives: 1
Potential false positives: 1
